In [16]:
import pandas as pd
import time

start_time = time.time()

df = pd.read_csv("group.csv", sep=",")
# print(df.value_counts())

bad_list = ["2", "111", "1", "261", "262", "250", "3", "4", "5",
             "6", "7", "8", "9", "10", "11", "102", "103", "104",
             "105", "106", "107", "108", "109", "113"]
bad_list = [int(i) for i in bad_list]

# print(bad_list)

# print(df.head())

# Коэффициент для обработки только определенного количества строк. Просто для себя.
k = 0
my_dict = {}

for now in df.values:
    # Список шагов, взятые из строки с индексом 0.
    list_steps = now[0].strip("[]").split(",")
    # Счетчик звонков, взятый из этой же строки с индексом 1.
    count = now[1]
    # print(list_steps, count)
    # Я брал только те шаги, где количество больше или равно 4.
    if len(list_steps) >= 4:
        list_steps = [int(i.strip("''")) for i in list_steps]
        # print(list_steps, count)
        # start, end - срезы для строк.
        start = -4
        end = -1
        # Пробегаем по каждой строке, пока модуль числа start будет меньше или равен длине списка шагов.
        while abs(start) <= len(list_steps):
            # first - список шагов, для которых проводится анализ. Длина списка == 3.
            first = tuple(list_steps[start:end])
            # second - шаг, на основании которого принимается решение о положительном или отрицательном исходе.
            second = list_steps[end]
            start -= 1
            end -= 1
            # print(f"first: {first}, second: {second}")
            # Дальше все это идет в словарь.
            # Ключи словаря - first.
            # Значения - список со счетчиком count. 
            # На месте индекса 0 - количество положительных завершений.
            # На месте индекса 1 - количество отрицательных завершений.
            # [+, -]: [good, bad]
            if first in my_dict:
                if second in bad_list:
                    my_dict[first][1] += count
                else:
                    my_dict[first][0] += count
            else:
                if second in bad_list:
                    my_dict[first] = [0, count]
                else:
                    my_dict[first] = [count, 0]
    # Обработал первые 1'000'000 значений.
    k += 1    
    if k == 1000000:
        break

# print(my_dict)

# Еще один словарь, для подсчета коэффициента для каждого из ключей (два предпоследних шага).
result_dict = dict()

for now in my_dict:
    # Шанс рассчитал так же, как и у тебя.
    chance = round(my_dict[now][1] / (my_dict[now][0] + my_dict[now][1]), 2)
    # Дальше словарь. Ключ - как и в прошлом словаре.
    # И уже значения - шанс того, насколько плохо закончится звонок.
    result_dict[now] = chance

print(result_dict)
print()
print(f"Длина словаря составляет: {len(result_dict)} значений.")
print(f"Время обработки {k} строк составило: {round(time.time() - start_time, 2)} сек.")


last_steps                                              count(phone_num)
['0','10','10','10']                                    1                   1
['250','102','2','109','3','40']                        1                   1
['250','102','2','109','261','93']                      1                   1
['250','102','2','109','3','10','111','40']             1                   1
['250','102','2','109','3','10','2','2','111','2','2']  1                   1
                                                                           ..
['111','111','261','111','3','105','111','111','261']   1                   1
['111','111','261','111','3','105','111','2']           1                   1
['111','111','261','111','3','105','111']               1                   1
['111','111','261','111','3','105','2','101','2']       1                   1
['99','99']                                             290                 1
Length: 15240192, dtype: int64
{(83, 3, 2): 1.0, (4, 4, 2): 0.91, (25